# Text Lab
In this lab, I took 3,150 paper abstracts from pubmed.gov that were related to brain connectivity research and analyzed the connectivity of the regions based on the literature. I'm interested in analyzing and displaying this data because I'm intrigued by the mysterious ways the brain functions.

In this notebook, I will clean and analyze data from the abstracts.

In [1]:
import json
import string

# import numpy as np
import pandas as pd

import nltk

I downloaded the abstracts from pubmed.gov in .txt form.

In [53]:
with open("pubmed_result.txt", "r") as f:
    text = f.read()
   

Now that the file is read, I will separate the text by abstract. To do so, I used the regex tokenizer:

In [3]:
# from nltk import WhitespaceTokenizer
from nltk import RegexpTokenizer

In [4]:
re_tokenizer = RegexpTokenizer("^(\n\n[0-9]+\. [a-z,A-Z].+?\n\n)")
sections = re_tokenizer.tokenize(text)

In [5]:
#Check that there are 3150 abstracts (none are not counted)
len(sections)

3150

In [6]:
abstract_by_headers = list(zip(re_tokenizer.tokenize(text), re_tokenizer.span_tokenize(text)))
abstract_by_headers

[('\n\n1. Proc Natl Acad Sci U S A. 2017 Oct 23. pii: 201712928. doi:\n10.1073/pnas.1712928114. [Epub ahead of print]\n\n',
  (0, 113)),
 ('\n\n2. J Neurophysiol. 2017 Oct 25:jn.00163.2017. doi: 10.1152/jn.00163.2017. [Epub\nahead of print]\n\n',
  (2643, 2742)),
 ('\n\n3. J Neurosci. 2017 Oct 24. pii: 2033-17. doi: 10.1523/JNEUROSCI.2033-17.2017. [Epub\nahead of print]\n\n',
  (4962, 5066)),
 ('\n\n4. Neuroimage. 2017 Oct 21. pii: S1053-8119(17)30866-2. doi:\n10.1016/j.neuroimage.2017.10.039. [Epub ahead of print]\n\n',
  (8237, 8357)),
 ('\n\n5. Brain. 2017 Sep 1;140(9):2370-2380. doi: 10.1093/brain/awx169.\n\n',
  (11000, 11069)),
 ('\n\n6. Neuron. 2017 Oct 11;96(2):505-520.e7. doi: 10.1016/j.neuron.2017.09.033.\n\n',
  (13845, 13924)),
 ('\n\n7. Brain Imaging Behav. 2017 Oct 4. doi: 10.1007/s11682-017-9778-8. [Epub ahead of\nprint]\n\n',
  (15670, 15763)),
 ('\n\n8. Neuroimage Clin. 2017 Sep 6;16:575-585. doi: 10.1016/j.nicl.2017.09.004.\neCollection 2017.\n\n',
  (18869, 18966)),


In [7]:
abstracts = []

for i, abstract in enumerate(abstract_by_headers):
    a = [abstract[0], [abstract[1][1], 0]]
    try: 
        a[1][1] = abstract_by_headers[i+1][1][0]
    except:
        pass
    abstracts.append(a)
    
abstracts[0:10]

[['\n\n1. Proc Natl Acad Sci U S A. 2017 Oct 23. pii: 201712928. doi:\n10.1073/pnas.1712928114. [Epub ahead of print]\n\n',
  [113, 2643]],
 ['\n\n2. J Neurophysiol. 2017 Oct 25:jn.00163.2017. doi: 10.1152/jn.00163.2017. [Epub\nahead of print]\n\n',
  [2742, 4962]],
 ['\n\n3. J Neurosci. 2017 Oct 24. pii: 2033-17. doi: 10.1523/JNEUROSCI.2033-17.2017. [Epub\nahead of print]\n\n',
  [5066, 8237]],
 ['\n\n4. Neuroimage. 2017 Oct 21. pii: S1053-8119(17)30866-2. doi:\n10.1016/j.neuroimage.2017.10.039. [Epub ahead of print]\n\n',
  [8357, 11000]],
 ['\n\n5. Brain. 2017 Sep 1;140(9):2370-2380. doi: 10.1093/brain/awx169.\n\n',
  [11069, 13845]],
 ['\n\n6. Neuron. 2017 Oct 11;96(2):505-520.e7. doi: 10.1016/j.neuron.2017.09.033.\n\n',
  [13924, 15670]],
 ['\n\n7. Brain Imaging Behav. 2017 Oct 4. doi: 10.1007/s11682-017-9778-8. [Epub ahead of\nprint]\n\n',
  [15763, 18869]],
 ['\n\n8. Neuroimage Clin. 2017 Sep 6;16:575-585. doi: 10.1016/j.nicl.2017.09.004.\neCollection 2017.\n\n',
  [18966, 22773

Now that we have the 3150 separated, let's disregard the headers and only focus on the abstract body itself

In [8]:
# split into the two parts of the text
sectionized_text = []

for abstract in abstracts:
    abstract_text = text[abstract[1][0]: abstract[1][1]-1]
    sectionized_text.append(abstract_text.replace("\n", " ").replace("\r", ""))

In [9]:
sectionized_text[0]

'Organizing principles for the cerebral cortex network of commissural and association connections.  Swanson LW(1), Hahn JD(2), Sporns O(3)(4).  Author information:   (1)Department of Biological Sciences, University of Southern California, Los Angeles, CA 90089; larryswanson10@gmail.com. (2)Department of Biological Sciences, University of Southern California, Los Angeles, CA 90089. (3)Department of Psychological and Brain Sciences, Indiana University, Bloomington, IN 47405. (4)Indiana University Network Science Institute, Indiana University, Bloomington, IN 47405.  Cognition is supported by a network of axonal connections between gray matter regions within and between right and left cerebral cortex. Global organizing principles of this circuitry were examined with network analysis tools applied to monosynaptic association (within one side) and commissural (between sides) connections between all 77 cortical gray matter regions in each hemisphere of the rat brain. The analysis used 32,350

Now that we've got the separate abstracts, let's tokenize each one by word. 

In [10]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

In [11]:
"""
This is just the code from above that tokenizes the sentence into 
words, then  and removes stop words, wrapped 
in a function.
"""
def tokenize_sentence(sentence):
    word_tokens = nltk.word_tokenize(sentence)
    for i in range(len(word_tokens)-1, -1, -1):
        word = word_tokens[i].lower()
        deleted = False
        if word in stopwords.words('english'):
            word_tokens.pop(i)
            deleted = True
        if len(word) <= 2 and not deleted: 
            word_tokens.pop(i)
            deleted = True
        if not deleted:
            word_tokens[i] = lemmatizer.lemmatize(word).lower()
                
    return word_tokens

In [12]:
tokenize_sentence(sectionized_text[0])

['organizing',
 'principle',
 'cerebral',
 'cortex',
 'network',
 'commissural',
 'association',
 'connection',
 'swanson',
 'hahn',
 'sporns',
 'author',
 'information',
 'department',
 'biological',
 'science',
 'university',
 'southern',
 'california',
 'los',
 'angeles',
 '90089',
 'larryswanson10',
 'gmail.com',
 'department',
 'biological',
 'science',
 'university',
 'southern',
 'california',
 'los',
 'angeles',
 '90089',
 'department',
 'psychological',
 'brain',
 'science',
 'indiana',
 'university',
 'bloomington',
 '47405',
 'indiana',
 'university',
 'network',
 'science',
 'institute',
 'indiana',
 'university',
 'bloomington',
 '47405',
 'cognition',
 'supported',
 'network',
 'axonal',
 'connection',
 'gray',
 'matter',
 'region',
 'within',
 'right',
 'left',
 'cerebral',
 'cortex',
 'global',
 'organizing',
 'principle',
 'circuitry',
 'examined',
 'network',
 'analysis',
 'tool',
 'applied',
 'monosynaptic',
 'association',
 'within',
 'one',
 'side',
 'commissural',

Now that each abstract is tokenized into words, we can count the occurrences of each brain region in each abstract. To do so, let's vectorize the data with the brain region terms.

In [13]:
brain_regions = ["frontal", "temporal", "parietal", "occipital", "limbic", "cerebral", "cerebrum", "cerebellum", "cerebellar", "thalamus", "hypothalamus", "hippocampus", "amygdala", "midbrain", "pons", "medula", "brainstem", "stem"]

def vectorize(word_tokens):
    vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for i in range(0, len(brain_regions)):
        word = brain_regions[i]
        if(i == 5):
            if(word in word_tokens or brain_regions[i+1] in word_tokens):
                vector[5] = 1
        elif(i == 7):
            if(word in word_tokens or brain_regions[i+1] in word_tokens):
                vector[6] = 1
        elif(i == 16):
            if(word in word_tokens):
                vector[14] = 1
            else:
                if(brain_regions[i+1] in word_tokens):
                    for j in range(0, len(word_tokens)):
                        if(word_tokens[j] == brain_regions[i+1] and j != 0):
                            if(word_tokens[j-1] == 'brain'):
                                vector[14] = 1
        elif(i == 6 or i == 8 or i == 17):
            pass
        elif(i>8 and i <16):
            if word in word_tokens:
                vector[i-2] = 1 
        else:
            if word in word_tokens:
                vector[i] = 1
    
    return vector

In [14]:
final_text_format = []

for i, part in enumerate(sectionized_text):
    parts_in_word_tokens = tokenize_sentence(part)
    final_text_format.append(vectorize(parts_in_word_tokens))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


Since my goal is to analyze the connectivity between pairs of brain regions, I now compare the pairs with one another:

In [45]:
comparingMatrix = []

regions = ["frontal", "temporal", "parietal", "occipital", "limbic", "cerebrum", "cerebellum", "thalamus", "hypothalamus", "hippocampus", "amygdala", "midbrain", "pons", "medula", "brainstem"]

for i in range(0, len(regions)):
    for j in range(0, i):
        word1 = regions[i]
        word2 = regions[j]
        twoByTwo = [[word1, word2],[0,0],[0,0]]
        for abstract in final_text_format:
            if abstract[i] == 1:
                if abstract[j] == 1:
                    twoByTwo[1][0] += 1 #abstracts containing word1 and word2
                else:
                    twoByTwo[1][1] += 1 #abstracts containing word1 but not word2
            else:
                if abstract[j] == 1:
                    twoByTwo[2][0] += 1 #abstracts containing word1 and word2
                else:
                    twoByTwo[2][1] += 1 #abstracts containing neither word1 nor word2
        comparingMatrix.append(twoByTwo)
comparingMatrix

[[['temporal', 'frontal'], [138, 336], [211, 2465]],
 [['parietal', 'frontal'], [121, 154], [228, 2647]],
 [['parietal', 'temporal'], [134, 141], [340, 2535]],
 [['occipital', 'frontal'], [43, 74], [306, 2727]],
 [['occipital', 'temporal'], [59, 58], [415, 2618]],
 [['occipital', 'parietal'], [48, 69], [227, 2806]],
 [['limbic', 'frontal'], [39, 155], [310, 2646]],
 [['limbic', 'temporal'], [49, 145], [425, 2531]],
 [['limbic', 'parietal'], [15, 179], [260, 2696]],
 [['limbic', 'occipital'], [4, 190], [113, 2843]],
 [['cerebrum', 'frontal'], [49, 273], [300, 2528]],
 [['cerebrum', 'temporal'], [61, 261], [413, 2415]],
 [['cerebrum', 'parietal'], [33, 289], [242, 2586]],
 [['cerebrum', 'occipital'], [23, 299], [94, 2734]],
 [['cerebrum', 'limbic'], [19, 303], [175, 2653]],
 [['cerebellum', 'frontal'], [35, 184], [314, 2617]],
 [['cerebellum', 'temporal'], [23, 196], [451, 2480]],
 [['cerebellum', 'parietal'], [24, 195], [251, 2680]],
 [['cerebellum', 'occipital'], [7, 212], [110, 2821]]

And now I analyze the pairs to find the chi-squared or XSquared value (representing connectivity) for each pair:

In [46]:
# expected values
XSquaredConnectivity = []
# expectedValues = []
for pair in comparingMatrix:
    if 'medula' not in pair[0]:
        a = pair[1][0]
        b = pair[2][0]
        c = pair[1][1]
        d = pair[2][1]
        R1 = a + b
        R2 = c + d
        C1 = a + c
        C2 = b + d
        N = C1 + C2

        E_a = (R1*C1)/N
        E_b = (R1*C2)/N
        E_c = (R2*C1)/N
        E_d = (R2*C2)/N
        
        XSquared = ((a-E_a)**2)/E_a + ((b-E_b)**2)/E_b + ((c-E_c)**2)/E_c + ((d-E_d)**2)/E_d 
        labelledMatrix = [pair[0], XSquared]
        XSquaredConnectivity.append(labelledMatrix)
    else:
        XSquaredConnectivity.append([pair[0], 0])

In [47]:
XSquaredConnectivity

[[['temporal', 'frontal'], 184.2028777174953],
 [['parietal', 'frontal'], 331.45484801006523],
 [['parietal', 'temporal'], 267.3605276164933],
 [['occipital', 'frontal'], 81.292802109426],
 [['occipital', 'temporal'], 118.9843176756613],
 [['occipital', 'parietal'], 159.05895291651967],
 [['limbic', 'frontal'], 17.086865045536964],
 [['limbic', 'temporal'], 16.85875892962213],
 [['limbic', 'parietal'], 0.25851962739551754],
 [['limbic', 'occipital'], 1.5783992097164279],
 [['cerebrum', 'frontal'], 6.233846003442049],
 [['cerebrum', 'temporal'], 4.25979455270221],
 [['cerebrum', 'parietal'], 1.0376387355181644],
 [['cerebrum', 'occipital'], 11.788980769795765],
 [['cerebrum', 'limbic'], 0.04134361061945554],
 [['cerebellum', 'frontal'], 5.741608923149829],
 [['cerebellum', 'temporal'], 3.803885898004479],
 [['cerebellum', 'parietal'], 1.467270974910348],
 [['cerebellum', 'occipital'], 0.17654620270818996],
 [['cerebellum', 'limbic'], 2.5570161921949093],
 [['cerebellum', 'cerebrum'], 14

Now I add in the adjacency of the brain regions:

In [48]:
dfAdj = pd.read_csv('regionAdjacent.csv')
dfAdj = dfAdj.loc[:,'0':]
dfAdj

,0,1
0,"['temporal', 'frontal']",1.0
1,"['parietal', 'frontal']",1.0
2,"['parietal', 'temporal']",1.0
3,"['occipital', 'frontal']",0.0
4,"['occipital', 'temporal']",1.0
5,"['occipital', 'parietal']",1.0
6,"['limbic', 'frontal']",1.0
7,"['limbic', 'temporal']",1.0
8,"['limbic', 'parietal']",1.0
9,"['limbic', 'occipital']",1.0


In [49]:
for i, entry in enumerate(XSquaredConnectivity):
    entry.append(dfAdj['1'][i])

In [50]:
XSquaredConnectivity

[[['temporal', 'frontal'], 184.2028777174953, 1.0],
 [['parietal', 'frontal'], 331.45484801006523, 1.0],
 [['parietal', 'temporal'], 267.3605276164933, 1.0],
 [['occipital', 'frontal'], 81.292802109426, 0.0],
 [['occipital', 'temporal'], 118.9843176756613, 1.0],
 [['occipital', 'parietal'], 159.05895291651967, 1.0],
 [['limbic', 'frontal'], 17.086865045536964, 1.0],
 [['limbic', 'temporal'], 16.85875892962213, 1.0],
 [['limbic', 'parietal'], 0.25851962739551754, 1.0],
 [['limbic', 'occipital'], 1.5783992097164279, 1.0],
 [['cerebrum', 'frontal'], 6.233846003442049, 0.5],
 [['cerebrum', 'temporal'], 4.25979455270221, 0.5],
 [['cerebrum', 'parietal'], 1.0376387355181644, 0.5],
 [['cerebrum', 'occipital'], 11.788980769795765, 0.5],
 [['cerebrum', 'limbic'], 0.04134361061945554, 0.5],
 [['cerebellum', 'frontal'], 5.741608923149829, 0.0],
 [['cerebellum', 'temporal'], 3.803885898004479, 1.0],
 [['cerebellum', 'parietal'], 1.467270974910348, 0.0],
 [['cerebellum', 'occipital'], 0.17654620270

In [51]:
df1 = pd.DataFrame(XSquaredConnectivity, columns = ['Pair', 'XSquared', 'Adjacent'])
df1

,Pair,XSquared,Adjacent
0,"[temporal, frontal]",184.202878,1.0
1,"[parietal, frontal]",331.454848,1.0
2,"[parietal, temporal]",267.360528,1.0
3,"[occipital, frontal]",81.292802,0.0
4,"[occipital, temporal]",118.984318,1.0
5,"[occipital, parietal]",159.058953,1.0
6,"[limbic, frontal]",17.086865,1.0
7,"[limbic, temporal]",16.858759,1.0
8,"[limbic, parietal]",0.258520,1.0
9,"[limbic, occipital]",1.578399,1.0


In [52]:
df1.to_csv('brainRegionPairs.csv')

In [41]:
regions

['frontal',
 'temporal',
 'parietal',
 'occipital',
 'limbic',
 'cerebrum',
 'cerebellum',
 'thalamus',
 'hypothalamus',
 'hippocampus',
 'amygdala',
 'midbrain',
 'pons',
 'medula',
 'brainstem']

In [42]:
classedRegions = []
for i, region in enumerate(regions):
    classedRegion = [region, '']
    if(i < 6):
        classedRegion[1] = 'cerebrum'
    elif(i == 6):
        classedRegion[1] = 'hindbrain'
    elif(i > 6 and i < 11):
        classedRegion[1] = 'limbic system'
    else:
        classedRegion[1] = 'brainstem'
    classedRegions.append(classedRegion)
    
classedRegions

[['frontal', 'cerebrum'],
 ['temporal', 'cerebrum'],
 ['parietal', 'cerebrum'],
 ['occipital', 'cerebrum'],
 ['limbic', 'cerebrum'],
 ['cerebrum', 'cerebrum'],
 ['cerebellum', 'hindbrain'],
 ['thalamus', 'limbic system'],
 ['hypothalamus', 'limbic system'],
 ['hippocampus', 'limbic system'],
 ['amygdala', 'limbic system'],
 ['midbrain', 'brainstem'],
 ['pons', 'brainstem'],
 ['medula', 'brainstem'],
 ['brainstem', 'brainstem']]

In [43]:
df2 = pd.DataFrame(classedRegions, columns=['Region', 'Class'])
df2

,Region,Class
0,frontal,cerebrum
1,temporal,cerebrum
2,parietal,cerebrum
3,occipital,cerebrum
4,limbic,cerebrum
5,cerebrum,cerebrum
6,cerebellum,hindbrain
7,thalamus,limbic system
8,hypothalamus,limbic system
9,hippocampus,limbic system


In [44]:
df2.to_csv("classedRegions.csv")